# House Price Predictor

In [72]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

In [73]:
# -----------------------------
# 1. Load the dataset
# -----------------------------
# Replace 'house_prices.csv' with the path to your dataset.
data = pd.read_csv('train.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,50,RL,51.0,4712,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,8,2006,WD,Abnorml,121600
996,997,20,RL,NaN,10659,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2006,COD,Normal,136500
997,998,20,RL,NaN,11717,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2009,WD,Normal,185000
998,999,30,RM,60.0,9786,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,91000


In [74]:
# -----------------------------
# 2. Data Cleaning
# -----------------------------
# Select only numeric columns that have no missing data.
numeric_cols = data.select_dtypes(include=[np.number]).columns
clean_numeric_cols = [col for col in numeric_cols if data[col].isna().sum() == 0]
data_clean = data[clean_numeric_cols]

# Ensure that the target column 'price' is present.
if 'SalePrice' not in data_clean.columns:
    raise ValueError("The target column 'price' is not present in the complete numeric data.")

In [75]:
# -----------------------------
# 3. Feature Selection
# -----------------------------
# Compute the correlation matrix using only the cleaned numeric data.
corr_matrix = data_clean.corr()

# Compute absolute correlations of features with the target and drop the target itself.
target_corr = corr_matrix['SalePrice'].drop('SalePrice').abs().sort_values(ascending=False)

# Select only the top 4 features with the highest correlation with 'SalesPrice'
top4_features = target_corr.head(4).index
top10_features = target_corr.head(10).index 

print("Selected top 10 features:", list(top10_features))

# Define input features (X) and target variable (y).
X = data_clean[top10_features].values
y = data_clean['SalePrice'].values.reshape(-1, 1)

Selected top 10 features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearBuilt', 'YearRemodAdd']


In [76]:
# -----------------------------
# 4. Data Preprocessing
# -----------------------------
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features to improve training stability.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [77]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [78]:
# -----------------------------
# 5. Define the Neural Network Model
# -----------------------------
class HousePriceModel(nn.Module):
    def __init__(self, input_size):
        super(HousePriceModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),  # Input layer
            nn.ReLU(),
            nn.Linear(64, 32),  # Hidden layer
            nn.ReLU(),
            nn.Linear(32, 1)  # Output layer
        )

    def forward(self, x):
        return self.model(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Ensure the model dynamically adapts to feature selection
input_size = X_train.shape[1]  # Number of selected features
model = HousePriceModel(input_size).to(device)

# Use GPU if available
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HousePriceModel(input_size=X_train.shape[1]).to(device)

In [79]:
X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [80]:
# -----------------------------
# 6. Set Up Loss Function and Optimizer
# -----------------------------
criterion = nn.SmoothL1Loss() 
optimizer = optim.AdamW(model.parameters(), lr=0.0001)  

# -----------------------------
# 7. Train the Model
# -----------------------------
num_epochs = 5000

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 100/5000, Loss: 170829.9062
Epoch 200/5000, Loss: 188163.9375
Epoch 300/5000, Loss: 175265.1562
Epoch 400/5000, Loss: 186907.2812
Epoch 500/5000, Loss: 182872.6719
Epoch 600/5000, Loss: 158104.3906
Epoch 700/5000, Loss: 136545.9844
Epoch 800/5000, Loss: 142302.3594
Epoch 900/5000, Loss: 117887.0156
Epoch 1000/5000, Loss: 97650.0156
Epoch 1100/5000, Loss: 77423.7500
Epoch 1200/5000, Loss: 70760.2422
Epoch 1300/5000, Loss: 72302.5469
Epoch 1400/5000, Loss: 69178.9531
Epoch 1500/5000, Loss: 60314.1250
Epoch 1600/5000, Loss: 57050.6953
Epoch 1700/5000, Loss: 59514.3281
Epoch 1800/5000, Loss: 58966.9258
Epoch 1900/5000, Loss: 57680.9766
Epoch 2000/5000, Loss: 41634.1523
Epoch 2100/5000, Loss: 53402.5547
Epoch 2200/5000, Loss: 52758.7188
Epoch 2300/5000, Loss: 40522.7344
Epoch 2400/5000, Loss: 31722.6367
Epoch 2500/5000, Loss: 45103.4922
Epoch 2600/5000, Loss: 46896.6875
Epoch 2700/5000, Loss: 38593.3242
Epoch 2800/5000, Loss: 39073.6719
Epoch 2900/5000, Loss: 25839.3379
Epoch 3000/500

In [81]:
# 8. Evaluate the Model
# -----------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor).item()
    print("Test Mean Squared Error:", test_loss)

predictions_np = predictions.cpu().numpy().flatten()
mse = mean_squared_error(y_test.flatten(), predictions_np)
print("Test MSE (scikit-learn):", mse)
# Test Mean Squared Error: 935741376.0

Test Mean Squared Error: 20505.193359375
Test MSE (scikit-learn): 836243968.0


1000
Test Mean Squared Error: 110692.2421875
Test MSE (scikit-learn): 15730100224.0

5000
Test Mean Squared Error: 20505.193359375
Test MSE (scikit-learn): 836243968.0

In [82]:
test_data = pd.read_csv("test.csv")

test_data_clean = test_data.select_dtypes(include=[np.number])
test_data_clean = test_data_clean.fillna(test_data_clean.median())
test_data_scaled = scaler.transform(test_data_clean[top10_features])  # Use selected features only

# Convert to PyTorch tensor and move to device
X_test_final_tensor = torch.tensor(test_data_scaled, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    test_predictions = model(X_test_final_tensor).cpu().numpy().flatten()  # Convert to numpy array

#help with format
submission_df = pd.DataFrame({
    "ID": test_data["Id"].astype(int), 
    "SALEPRICE": test_predictions.astype(float)
})

# Save to CSV (format must be exact)
submission_df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")


Predictions saved to predictions.csv


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
